# Sudoku Minimal Test

A self-contained test that:
1. Generates a Sudoku puzzle
2. Creates a board image
3. Solves the puzzle
4. Verifies the solution is correct

In [ ]:
# Install dependencies if needed
!pip install z3-solver pillow matplotlib -q

In [ ]:
from z3 import *
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import random
import matplotlib.pyplot as plt
import os

print("Imports successful!")

## Step 1: Define Puzzle Generation Functions

In [ ]:
def generate_complete_grid(size):
    """Generate a complete, valid Sudoku grid using Z3."""
    box_size = 2 if size == 4 else 3
    
    X = [[Int(f"x_{i}_{j}") for j in range(size)] for i in range(size)]
    s = Solver()
    
    # Cell range constraints
    for i in range(size):
        for j in range(size):
            s.add(And(X[i][j] >= 1, X[i][j] <= size))
    
    # Row constraints
    for i in range(size):
        s.add(Distinct([X[i][j] for j in range(size)]))
    
    # Column constraints
    for j in range(size):
        s.add(Distinct([X[i][j] for i in range(size)]))
    
    # Box constraints
    for box_row in range(box_size):
        for box_col in range(box_size):
            cells = []
            for i in range(box_size):
                for j in range(box_size):
                    cells.append(X[box_row * box_size + i][box_col * box_size + j])
            s.add(Distinct(cells))
    
    # Add random constraints for variety
    random_cells = random.sample([(i, j) for i in range(size) for j in range(size)], min(3, size))
    for (i, j) in random_cells:
        val = random.randint(1, size)
        s.add(X[i][j] == val)
    
    if s.check() == sat:
        m = s.model()
        return [[m.evaluate(X[i][j]).as_long() for j in range(size)] for i in range(size)]
    else:
        return generate_complete_grid(size)  # Retry if random constraints conflict


def has_unique_solution(size, given_cells):
    """Check if puzzle has exactly one solution."""
    box_size = 2 if size == 4 else 3
    
    X = [[Int(f"x_{i}_{j}") for j in range(size)] for i in range(size)]
    s = Solver()
    
    for i in range(size):
        for j in range(size):
            s.add(And(X[i][j] >= 1, X[i][j] <= size))
    
    for (i, j), val in given_cells.items():
        s.add(X[i][j] == val)
    
    for i in range(size):
        s.add(Distinct([X[i][j] for j in range(size)]))
    
    for j in range(size):
        s.add(Distinct([X[i][j] for i in range(size)]))
    
    for box_row in range(box_size):
        for box_col in range(box_size):
            cells = [X[box_row * box_size + i][box_col * box_size + j] 
                     for i in range(box_size) for j in range(box_size)]
            s.add(Distinct(cells))
    
    if s.check() != sat:
        return False
    
    first_solution = s.model()
    block = [X[i][j] != first_solution.evaluate(X[i][j]) 
             for i in range(size) for j in range(size)]
    s.add(Or(block))
    
    return s.check() == unsat


def generate_puzzle(size):
    """Generate a puzzle by removing cells from a complete grid."""
    min_clues = 4 if size == 4 else 17
    
    solution = generate_complete_grid(size)
    puzzle = [row[:] for row in solution]
    given_cells = {(i, j): solution[i][j] for i in range(size) for j in range(size)}
    
    cells_to_try = [(i, j) for i in range(size) for j in range(size)]
    random.shuffle(cells_to_try)
    
    for (i, j) in cells_to_try:
        if len(given_cells) <= min_clues:
            break
        
        val = given_cells.pop((i, j))
        
        if has_unique_solution(size, given_cells):
            puzzle[i][j] = 0
        else:
            given_cells[(i, j)] = val
    
    return puzzle, solution

print("Puzzle generation functions defined!")

## Step 2: Generate a Puzzle

In [ ]:
print("Generating 4x4 Sudoku puzzle...")
puzzle, expected_solution = generate_puzzle(4)

print("\nPuzzle (0 = empty):")
for row in puzzle:
    print(row)

print("\nExpected Solution:")
for row in expected_solution:
    print(row)

clue_count = sum(1 for row in puzzle for cell in row if cell != 0)
print(f"\nClues given: {clue_count} / 16")

## Step 3: Create Board Image

In [ ]:
def make_sudoku_board(size):
    """Create a blank Sudoku board with grid lines."""
    grid = np.ones((900, 900))
    box_size = 2 if size == 4 else 3
    cell_size = 900 // size
    
    # Outer border
    for i in range(8):
        grid[i, :] = 0
        grid[-i-1, :] = 0
        grid[:, i] = 0
        grid[:, -i-1] = 0
    
    # Internal lines
    for i in range(1, size):
        pos = i * cell_size
        thickness = 6 if i % box_size == 0 else 2
        
        for j in range(-thickness, thickness):
            if 0 <= pos + j < 900:
                grid[pos + j, :] = 0
                grid[:, pos + j] = 0
    
    return grid


def make_board_full(size, puzzle):
    """Create board with digits using PIL."""
    grid = make_sudoku_board(size)
    img = Image.fromarray((grid * 255).astype(np.uint8), mode='L')
    draw = ImageDraw.Draw(img)
    
    cell_size = 900 // size
    
    # Try to load a font
    try:
        font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", cell_size * 2 // 3)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", cell_size * 2 // 3)
        except:
            font = ImageFont.load_default()
    
    for row in range(size):
        for col in range(size):
            digit = puzzle[row][col]
            if digit != 0:
                x = col * cell_size + cell_size // 2
                y = row * cell_size + cell_size // 2
                
                text = str(digit)
                bbox = draw.textbbox((0, 0), text, font=font)
                text_w = bbox[2] - bbox[0]
                text_h = bbox[3] - bbox[1]
                
                draw.text((x - text_w // 2, y - text_h // 2 - bbox[1]), text, fill=0, font=font)
    
    return np.array(img) / 255.0


# Generate and display the board image
board_image = make_board_full(4, puzzle)

# Save image
os.makedirs('./board_images', exist_ok=True)
img_path = './board_images/test_puzzle.png'
Image.fromarray((board_image * 255).astype(np.uint8), mode='L').save(img_path)
print(f"Board image saved to: {img_path}")

# Display
plt.figure(figsize=(6, 6))
plt.imshow(board_image, cmap='gray')
plt.title('Generated Sudoku Puzzle')
plt.axis('off')
plt.show()

## Step 4: Solve the Puzzle

In [ ]:
def solve_sudoku(size, given_cells):
    """Solve Sudoku using Z3."""
    box_size = 2 if size == 4 else 3
    
    X = [[Int(f"x_{i}_{j}") for j in range(size)] for i in range(size)]
    s = Solver()
    
    # Cell range
    for i in range(size):
        for j in range(size):
            s.add(And(X[i][j] >= 1, X[i][j] <= size))
    
    # Given cells
    for (i, j), val in given_cells.items():
        s.add(X[i][j] == val)
    
    # Row uniqueness
    for i in range(size):
        s.add(Distinct([X[i][j] for j in range(size)]))
    
    # Column uniqueness
    for j in range(size):
        s.add(Distinct([X[i][j] for i in range(size)]))
    
    # Box uniqueness
    for box_row in range(box_size):
        for box_col in range(box_size):
            cells = [X[box_row * box_size + i][box_col * box_size + j]
                     for i in range(box_size) for j in range(box_size)]
            s.add(Distinct(cells))
    
    if s.check() == sat:
        m = s.model()
        return [[m.evaluate(X[i][j]).as_long() for j in range(size)] for i in range(size)]
    return None


def solve_from_json(puzzle):
    """Solve from puzzle array where 0 = empty."""
    size = len(puzzle)
    given_cells = {(i, j): puzzle[i][j] 
                   for i in range(size) for j in range(size) 
                   if puzzle[i][j] != 0}
    return solve_sudoku(size, given_cells)


# Solve the puzzle
print("Solving puzzle...")
computed_solution = solve_from_json(puzzle)

print("\nComputed Solution:")
for row in computed_solution:
    print(row)

## Step 5: Verify Result

In [ ]:
# Compare computed vs expected
is_correct = computed_solution == expected_solution

print("\n" + "=" * 50)
if is_correct:
    print("RESULT: PASS - Solution is CORRECT")
else:
    print("RESULT: FAIL - Solution is INCORRECT")
    print("\nExpected:")
    for row in expected_solution:
        print(row)
    print("\nGot:")
    for row in computed_solution:
        print(row)
print("=" * 50)

## Summary Visualization

In [ ]:
# Side-by-side visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Puzzle
axes[0].imshow(board_image, cmap='gray')
axes[0].set_title('Puzzle', fontsize=14)
axes[0].axis('off')

# Solution (create image with solution)
solution_image = make_board_full(4, computed_solution)
axes[1].imshow(solution_image, cmap='gray')
axes[1].set_title('Solution', fontsize=14)
axes[1].axis('off')

result_text = "PASS" if is_correct else "FAIL"
color = 'green' if is_correct else 'red'
fig.suptitle(f'Test Result: {result_text}', fontsize=16, color=color, fontweight='bold')

plt.tight_layout()
plt.show()